# Segmentation to Information Exploration 2
Last time (in `seg2info_exploration.ipynb`), we figured out the transformations necessary to get from a semantically segmented camera image to a map in real (log-polar) coordinates. Here, we'll focus on putting those routines into a Python module and on performing interpolation better to reduce fringe-type artifacts at high distance.

In [ ]:
%load_ext autoreload
%autoreload 2

We'll put the reusable routines in `cv_experiments.seg2info`.

In [ ]:
import os

import cv_experiments.seg2info as s2i

## Setup
We'll start by loading the same data as in part 1:

In [ ]:
images = s2i.load_dirs("../representatives/segmentation/seginput", "../arctic_images_original_2/segmaps")
names = [image["name"] for image in images]
print(names)
s2i.plot_all(images, s2i.simple_composite, lambda fig: fig.subplots_adjust(hspace=-0.5))
s2i.plot_all(images, s2i.plot_mask, lambda fig: fig.subplots_adjust(hspace=-0.5))

## Categorical to continuous
Our main strategy here will be to allocate one floating-point channel for each of the possible integer values in the mask. Initially this will be a one-hot encoding, but as we perform all the transformations, interpolation will give rise to fractional values representing "iciness," "skyness," etc. Then, if desired, we can apply an argmax or something at the end to get back to discrete data. For now, we'll visualize this data structure as an RGB image where green is iciness, blue is wateriness, and red is everything else-iness.

A small complication: certain OpenCV things don't like images with more than four channels, so instead of a full one-hot encoding, we'll just use the four channels water, sky, ice, and rest.

In [ ]:
s2i.apply(s2i.one_hot_four, images, "segmap", "segcont")
s2i.plot_key(images, "segcont", lambda fig: fig.subplots_adjust(hspace=-0.5))

Let's also upscale the images so the transformations to follow are less lossy:

In [ ]:
s2i.apply(s2i.upscale, images, "segcont", "upscaled")
print(images[0]["upscaled"].shape)
s2i.plot_key(images, "upscaled", lambda fig: fig.subplots_adjust(hspace=-0.5))

Now let's go through the existing pipeline, just with interpolation.

Undistortion and horizon finding:

In [ ]:
s2i.apply(s2i.undistort, images, "upscaled", "undistorted")
s2i.apply(s2i.find_horizon, images, "undistorted", "horizon")
s2i.plot_all(images, lambda ax,img: s2i.plot_line(ax, img, "undistorted", "horizon"), lambda fig: fig.subplots_adjust(hspace=-0.5))

Rotating, adjusting, and cropping:

In [ ]:
s2i.apply(s2i.rotate_image, images, ["undistorted", "horizon"], ["rotated", "rot_scale", "orig_height"])
s2i.apply(s2i.adjust_and_crop, images, ["rotated", "horizon", "orig_height"], "adjusted")
s2i.plot_key(images, "adjusted")

Coordinate transformation:

In [ ]:
s2i.apply(s2i.camera_to_log_polar, images, ["adjusted", "rot_scale", "orig_height"], "logpolar")
s2i.plot_all(images, s2i.plot_log_polar, lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Okay, we're back where we were. Slightly better results due to the interpolation, but we've still got some issues. How about a blur right before the coordinate transformation:

In [ ]:
import cv2
kernel = [s2i.upscale_factor*4+1, s2i.upscale_factor*2+1]
s2i.apply(lambda img: cv2.blur(img, kernel), images, "adjusted", "blurred")
s2i.plot_key(images, "blurred")
s2i.apply(s2i.camera_to_log_polar, images, ["blurred", "rot_scale", "orig_height"], "logpolar-2")
s2i.plot_all(images, lambda ax,img: s2i.plot_log_polar(ax, img, "logpolar-2"), lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Nope, looks like we really have to sacrifice image quality to get blurring to reduce the fringing much. More ideas: erode/dilate the ice in the original image? Resize using deep learning fanciness?

Let's put the whole process here where we can mess with it, and then try a few things:

In [ ]:
import numpy as np
def horizon_blur(img, orig_height, horizon_height=s2i.upscale_factor*s2i.sky_buffer):
    img = cv2.blur(img, (s2i.upscale_factor*3, s2i.upscale_factor*3))
    res = img.copy()
    x = np.arange(s2i.upscale_factor*5, -1, -1)
    y = (x[0]-x)*50/x[0]+3
    x += horizon_height
    for i, (blur_height, blur_radius) in enumerate(zip(x, y)):
        if blur_radius < 1: continue
        res[:int(blur_height),:] = cv2.blur(img[:int(blur_height),:], (int(blur_radius), int(blur_radius*0.05+1)))
    return res

def whole_pipeline(img):
    interpolation_method = cv2.INTER_CUBIC
    img = s2i.one_hot_four(img)
    img = s2i.upscale(img, interpolation_method=interpolation_method)
    img = s2i.undistort(img, interpolation_method=interpolation_method)
    horizon = s2i.find_horizon(img)
    img, scale, height = s2i.rotate_image(img, horizon, interpolation_method=interpolation_method)
    img = s2i.adjust_and_crop(img, horizon, height, interpolation_method=interpolation_method)
    img = horizon_blur(img, height)
    img = horizon_blur(img, height)  # Repetition intentional
    img = s2i.camera_to_log_polar(img, scale, height, interpolation_method=interpolation_method)
    return img

s2i.apply(whole_pipeline, images, "segmap", "logpolar-3")
s2i.plot_all(images, lambda ax,img: s2i.plot_log_polar(ax, img, "logpolar-3"), lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Okay, that's somewhat better. Now let's convert to a single channel "iciness" map where lack of data is represented by `NaN`:

In [ ]:
s2i.apply(s2i.four_to_one, images, "logpolar-3", "single_out")
s2i.plot_all(images, lambda ax,img: s2i.plot_log_polar(ax, img, "single_out"), lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))

Not too bad.

In [ ]:
s2i.apply(s2i.whole_pipeline, images, "segmap", "logpolar_final")
s2i.plot_all(images, lambda ax,img: s2i.plot_log_polar(ax, img, "logpolar_final"), lambda fig: fig.subplots_adjust(hspace=-0.5, wspace=0.4))